In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler
import numpy as np
import torch.nn.functional as F
from torch.autograd import Variable

In [78]:
file_csv = pd.read_csv("stock_market_prediction/csv_data/reuters_news.csv")

# cleaning data by removing raws that has null values in the polarity, subjectivity, or prices columns 
file_csv.dropna(subset = ["polarity"], inplace=True)
file_csv.dropna(subset = ["subjectivity"], inplace=True)
file_csv.dropna(subset = ["prices"], inplace=True)

# getting the values of each column
polarity = file_csv["polarity"].values
subjectivity = file_csv["subjectivity"].values
prices = file_csv["prices"].values


# making pairwise 2d-array of the polarity and subjectivity arrays
polarity = np.array(polarity, dtype=np.float32)
subjectivity = np.array(subjectivity, dtype=np.float32)
prices = np.array(prices, dtype=np.float32)


# devide data into testing, and training, and validation 
x_val = np.array(list(zip(polarity[:5000], subjectivity[:5000])), dtype=np.float32)
y_val = np.array(prices[:5000], dtype=np.float32)

x_train = np.array(list(zip(polarity[5000:24329], subjectivity[5000:24329])), dtype=np.float32)
y_train = np.array(list(zip(prices[5000:24329], prices[5000:24329])), dtype=np.float32)    #had to change this for dataloader batch sizes equality, to avoid error
# y_train.reshape((-1,2))

x_test = np.array(list(zip(polarity[24329:], subjectivity[24329:])), dtype=np.float32)
y_test = np.array(prices[24329:], dtype=np.float32)


# creating the tensors to prepare data for the model
x_val_tensor = torch.from_numpy(x_val)
y_val_tensor = torch.from_numpy(y_val)
y_val_tensor = y_val_tensor.to(torch.float64)


# -----------------
x_train_tensor = torch.from_numpy(x_train)
# x_train_tensor = x_train_tensor.to(torch.float64)
y_train_tensor = torch.from_numpy(y_train)
# y_train_tensor.resize(-1,2)
# y_train_tensor.view(-1,2)
# y_train_tensor = y_train_tensor.to(torch.float64)
# -----------------


x_test_tensor = torch.from_numpy(x_test)
y_test_tensor = torch.from_numpy(y_test)
y_test_tensor = y_test_tensor.to(torch.float64)

# for i in x_train:
#     if len(i)<1:
# for i in x_train:
#     print(i)
# train_loader = torch.utils.data.DataLoader(x_train_tensor, batch_size=20)

# x_train_loader = torch.utils.data.DataLoader(x_train_tensor, 64, shuffle=False)
# iter(x_train_loader).next()

# training_data = np.array(list(zip(x_train, y_train)), dtype=np.float32)
# training_data = torch.from_numpy(training_data)
# x_train_loader = torch.utils.data.DataLoader(training_data, 64, shuffle=False)
# iter(x_train_loader).next()
dataloader = torch.utils.data.DataLoader((x_train_tensor, y_train_tensor) , batch_size = 64, shuffle = True)

C:\Users\thabe\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [79]:
# define the model
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = torch.nn.Linear(2, 500)
        self.fc2 = torch.nn.Linear(500, 1000)
        self.fc3 = torch.nn.Linear(1000, 500)
        self.fc4 = torch.nn.Linear(500, 50)
        self.fc5 = torch.nn.Linear(50, 1)
        
        self.drop = torch.nn.Dropout(0.2)
#         self.double()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = F.relu(self.fc2(x))
        x = self.drop(x)
        x = F.relu(self.fc3(x))
        x = self.drop(x)
        x = F.relu(self.fc4(x))
        out = self.fc5(x)
        return out
model = Network()

In [122]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
critrian = torch.nn.L1Loss()

In [124]:
train_loss = 0.0
valid_loss = 0.0
for epoch in range(10000):
    model.train()
    
    for batch in dataloader:
        x, y = batch
        
#         delete the second column from the tensor y
        y = y.numpy()
        y = np.delete(y, 1, 1)
        y = torch.from_numpy(y)
        
#         print(y)
#         break
#     break
#         print(y[0][0])
        # train model
        optimizer.zero_grad()

        output = model(x)
        loss = critrian(output, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

# #       break
# #   break
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
            epoch, 
            train_loss))

    
#     inputs = Variable(torch.from_numpy(x_train))
#     labels = Variable(y_train_tensor)
    
# #     ---test----
#     optimizer.zero_grad()
#     output = model(inputs)
#     loss = critrian(output, labels.float())
#     loss.backward()
#     optimizer.step()
#     train_loss += loss.item()
    
    # train model
#     optimizer.zero_grad()
        
#     output = model(x_test_tensor.float())
#     loss = critrian(output, y_test_tensor.float())
#     loss.backward()
#     optimizer.step()
#     train_loss += loss.item()
    
#     model.eval()
#     output = model(x_test_tensor.float())
#     loss = critrian(output, y_test_tensor.float())
#     valid_loss += loss.item()

#     if (10000%(epoch+1) == 0):
#             print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
#             epoch, 
#             train_loss,
#             valid_loss
#             ))

#     for data, target in enumerate(x_train_tensor,y_train_tensor):
#         print(data.unsqueeze(dim=0))
#         optimizer.zero_grad()
        
#         output = model(data)
#         loss = critrian(output, target)
#         critrian.backward()
#         optimizer.step()
#         train_loss = train_loss + loss.item()
        
        



Epoch: 0 	Training Loss: 0.594454
Epoch: 1 	Training Loss: 190.130099
Epoch: 2 	Training Loss: 190.724353
Epoch: 3 	Training Loss: 191.318507
Epoch: 4 	Training Loss: 191.912561
Epoch: 5 	Training Loss: 381.447794
Epoch: 6 	Training Loss: 382.041648
Epoch: 7 	Training Loss: 382.635403
Epoch: 8 	Training Loss: 383.229057
Epoch: 9 	Training Loss: 383.822611
Epoch: 10 	Training Loss: 384.416065
Epoch: 11 	Training Loss: 385.009420
Epoch: 12 	Training Loss: 574.543965
Epoch: 13 	Training Loss: 764.078404
Epoch: 14 	Training Loss: 764.671459
Epoch: 15 	Training Loss: 954.205700
Epoch: 16 	Training Loss: 954.798554
Epoch: 17 	Training Loss: 1144.332596
Epoch: 18 	Training Loss: 1333.866547
Epoch: 19 	Training Loss: 1523.400391
Epoch: 20 	Training Loss: 1523.992845
Epoch: 21 	Training Loss: 1524.585199
Epoch: 22 	Training Loss: 1714.118754
Epoch: 23 	Training Loss: 1714.710908
Epoch: 24 	Training Loss: 1904.244248
Epoch: 25 	Training Loss: 2093.777497
Epoch: 26 	Training Loss: 2283.310639


KeyboardInterrupt: 